# 📘 Notebook Summary – 01_Data_Collection.ipynb
This notebook is the data collection pipeline for a broader music analytics project. It automates the process of gathering Billboard Hot 100 songs from 1959 to 2024, along with their lyrics, by combining web scraping and asynchronous API requests. The workflow is designed for performance, maintainability, and scalability.

## Key Features:
* **Chart Scraping:** Scrapes the Billboard Year-End Hot 100 list for each year using Wikipedia.

* **Async Lyrics Fetching:** Uses aiohttp to fetch lyrics from the lyrics.ovh API with intelligent title/artist variations and retry logic to maximize success rate.

* **Caching System:** Implements a JSON-based caching mechanism to avoid redundant API calls and improve performance across sessions.

* **Error Handling & Logging:** Gracefully handles missing data and logs failed entries for analysis.

* **Modular Functions:** Functions are clearly organized by role (scraping, cleaning, API calls, caching, orchestration), making the code reusable and extensible.

This notebook sets the foundation for subsequent analysis — including sentiment, word frequency, topic modeling, and visual storytelling based on lyrics trends.

## 📦 Imports & Setup

This section imports all the essential libraries used throughout the notebook:

* **Web Scraping & Requests**: requests, BeautifulSoup – to scrape Billboard Hot 100 chart data from Wikipedia.

* **Async I/O & Networking**: aiohttp, asyncio, nest_asyncio – to enable asynchronous fetching of lyrics from an API for improved performance in a Jupyter environment.

* **Data Handling**: pandas, numpy, json, os – for efficient data manipulation, caching, and storage.

* **Text Processing & NLP**: re, nltk, TextBlob, WordNetLemmatizer, stopwords – for cleaning, tokenizing, and analyzing song lyrics.

* **Visualization**: matplotlib, seaborn, WordCloud – to visualize insights such as most common words and trends.

* **Utility**: tqdm.asyncio – for progress bars in asynchronous loops, and logging for monitoring execution.

* **nest_asyncio.apply**() is called to allow nested event loops for Google Colab, enabling smooth use of asyncio.run() inside the notebook.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import aiohttp
import asyncio
import re
import urllib.parse
import time
import logging
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio
import os
import json
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
from nltk import ngrams
import zipfile

# Applying nest_asyncio for notebook compatibility
nest_asyncio.apply()
logging.basicConfig(level=logging.INFO)

## Creating function to scrape data from Billboard hot-100 year end pages on wikipedia

The function parses the HTML table on the wikipedia page using ***BeautifulSoup***, handles cases where artist names are omitted due to rowspans (if consecutive songs have the same artist), and returns a clean pandas DataFrame with the song's rank, title, and artist.

In [ ]:
# --------------------------
# WEB SCRAPING FUNCTIONS
# --------------------------

def scrape_hot_100(year):
    """Scrape Billboard Hot 100 for given year"""
    url = f"https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{year}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        logging.error(f"Error fetching {year}: {str(e)}")
        return None

    table = soup.find('table', {'class': 'wikitable sortable'})
    if table is None:
        logging.warning(f"Hot 100 table not found for {year}")
        return None

    data = []
    current_artist = None

    for row in table.find_all('tr')[1:]:  # Skip header
        cols = row.find_all('td')

        if len(cols) == 3:  # Full row with artist
            try:
                rank = int(cols[0].text.strip())
                title = cols[1].text.strip().strip('"')
                current_artist = cols[2].text.strip()
                data.append([rank, title, current_artist])
            except ValueError:
                continue

        elif len(cols) == 2 and current_artist:  # Rowspan artist case
            try:
                rank = int(cols[0].text.strip())
                title = cols[1].text.strip().strip('"')
                data.append([rank, title, current_artist])
            except ValueError:
                continue

    if not data:
        return None

    df = pd.DataFrame(data, columns=["Rank", "Title", "Artist"])
    return df.set_index("Rank")


## 🎵 Lyric Fetching Functions

This section defines asynchronous functions to fetch song lyrics from the lyrics.ovh API ( [website](https://lyricsovh.docs.apiary.io/#) / [github](https://github.com/NTag/lyrics.ovh)). It includes:



*   **clean_artist_name()**: Cleans artist names by removing extra punctuation, featured artists, and brackets to improve match accuracy.

*   **fetch_lyrics()**: Sends asynchronous API requests to retrieve lyrics for a given artist and song title.

*   **fetch_with_retries()**: Tries multiple variations of artist and song title combinations to maximize the chance of a successful match, with optional retries and delays for robustness.

These functions work together to collect lyrics data while handling naming inconsistencies common in music metadata.

In [ ]:
# --------------------------
# LYRIC FETCHING FUNCTIONS
# --------------------------

def clean_artist_name(artist):
    """Optimized artist cleaning function"""
    if not isinstance(artist, str):
        return ""

    # Remove content in parentheses and brackets
    artist = re.sub(r'\([^)]*\)|\[[^\]]*\]', '', artist)

    # Handle common patterns with a single regex
    match = re.search(r'^([^,;&/]+?)(?:\s*(?:,|&|and|featuring?|ft\.?|with|/|x)\s|$)', artist, flags=re.IGNORECASE)
    return match.group(1).strip() if match else artist.strip()

async def fetch_lyrics(session, artist, title, timeout=8):
    """Fetch lyrics with optimized parameters"""
    artist_encoded = urllib.parse.quote(artist)
    title_encoded = urllib.parse.quote(title)
    url = f"https://api.lyrics.ovh/v1/{artist_encoded}/{title_encoded}"

    try:
        async with session.get(url, timeout=timeout) as response:
            if response.status == 200:
                data = await response.json()
                return data.get('lyrics', '')
            elif response.status == 404:
                return ''  # Known missing
    except (aiohttp.ClientError, asyncio.TimeoutError):
        return ''
    return ''

async def fetch_with_retries(session, artist, title, max_retries=3):
    """Optimized with precomputed variations"""
    # Clean artist name once
    primary_artist = clean_artist_name(artist)

    # Precompute title variations
    base_title = re.sub(r'\([^)]*\)', '', title).strip()
    base_title_no_punct = re.sub(r'[\!\?\.\,\']', '', base_title)
    base_title_no_feat = title.split(' (')[0].strip()

    # Prepare all variations upfront
    variations = [
        (primary_artist, title),
        (primary_artist, base_title),
        (primary_artist, base_title_no_feat),
        (artist.split(',')[0].strip(), title),
        (artist, title),
        (primary_artist, f"{base_title} (feat. ...)"),
        (primary_artist, base_title_no_punct),
    ]

    # Add extra variations for problematic titles
    if any(char in title for char in ['!', '?', '.', ',', "'"]):
        variations.append((artist, base_title_no_punct))
        variations.append((primary_artist, base_title_no_punct))

    # Try variations without delay first
    for try_artist, try_title in variations[:max_retries]:
        lyrics = await fetch_lyrics(session, try_artist, try_title)
        if lyrics:
            return lyrics

    # Add delays only for remaining retries
    for i, (try_artist, try_title) in enumerate(variations[max_retries:max_retries*2]):
        await asyncio.sleep(0.2 * (i + 1))
        lyrics = await fetch_lyrics(session, try_artist, try_title)
        if lyrics:
            return lyrics

    return ''

##💾 Lyrics Caching System
This class implements a JSON-based cache to store and reuse fetched lyrics, reducing redundant API calls and improving efficiency. Key features include:

*   Persistent storage in lyrics_cache.json.

*   Efficient lookup using normalized artist and song title keys.

*   Periodic auto-saving every 50 entries to avoid frequent file I/O.

This helps speed up lyric retrieval in repeated or large-scale scraping runs.

In [ ]:
# --------------------------
# CACHE SYSTEM FOR REUSABLE RESULTS
# --------------------------

class LyricsCache:
    def __init__(self, cache_file='lyrics_cache.json'):
        self.cache_file = cache_file
        self.cache = self.load_cache()

    def load_cache(self):
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r') as f:
                    return json.load(f)
            except:
                return {}
        return {}

    def save_cache(self):
        with open(self.cache_file, 'w') as f:
            json.dump(self.cache, f)

    def get_key(self, artist, title):
        return f"{clean_artist_name(artist)}|||{title.lower().strip()}"

    def get(self, artist, title):
        return self.cache.get(self.get_key(artist, title), None)

    def set(self, artist, title, lyrics):
        key = self.get_key(artist, title)
        self.cache[key] = lyrics
        # Save periodically rather than on every set
        if len(self.cache) % 50 == 0:
            self.save_cache()

    def __len__(self):
        return len(self.cache)

## ⚙️ Optimized Main Execution Pipeline
This section orchestrates the full scraping and lyric-fetching workflow across multiple years of Billboard Hot 100 data. Key features include:

### process_year():

*   Scrapes chart data for a given year.

*   Retrieves cached lyrics where available.

*   Uses asynchronous batch requests to fetch missing lyrics concurrently.

*   Writes results to a CSV file and logs any failed fetches.

### main():

*   Coordinates the end-to-end process for multiple years.

*   Manages connection pooling via aiohttp for performance.

*   Persists the lyrics cache and logs missing entries for later review.


The workflow is designed for speed and efficiency, using async I/O, intelligent caching, and batch processing to handle large-scale data collection and enrichment.

In [ ]:
# --------------------------
# OPTIMIZED MAIN EXECUTION
# --------------------------

async def process_year(year, cache, session, concurrency=50):
    """Process a single year with caching and optimized fetching"""
    print(f"\n{'='*40}\nProcessing {year}\n{'='*40}")
    start_time = time.time()

    # Scrape Billboard data
    billboard_df = scrape_hot_100(year)
    if billboard_df is None:
        print(f"⚠️  No data found for {year}")
        return None, []

    print(f"Found {len(billboard_df)} songs for {year}")

    # Initialize lyrics column
    billboard_df['Lyrics'] = ''

    # Check cache first
    cached_count = 0
    for idx, row in billboard_df.iterrows():
        cache_key = cache.get_key(row['Artist'], row['Title'])
        if cache_key in cache.cache:
            billboard_df.at[idx, 'Lyrics'] = cache.cache[cache_key]
            cached_count += 1

    print(f"🚀 {cached_count}/{len(billboard_df)} lyrics from cache")

    # Prepare tasks for missing lyrics
    tasks = []
    for idx, row in billboard_df.iterrows():
        if not billboard_df.at[idx, 'Lyrics']:
            tasks.append(
                fetch_with_retries(session, row['Artist'], row['Title'])
            )

    # Process in batches for better memory management
    results = []
    batch_size = concurrency * 5  # Process in larger batches
    for i in range(0, len(tasks), batch_size):
        batch = tasks[i:i+batch_size]
        batch_results = await tqdm_asyncio.gather(
            *batch,
            desc=f"Fetching {year} lyrics",
            unit="song"
        )
        results.extend(batch_results)

    # Update DataFrame and cache
    result_idx = 0
    failed_entries = []
    for idx, row in billboard_df.iterrows():
        if not billboard_df.at[idx, 'Lyrics']:
            lyrics = results[result_idx]
            result_idx += 1

            if lyrics:
                billboard_df.at[idx, 'Lyrics'] = lyrics
                cache.set(row['Artist'], row['Title'], lyrics)
            else:
                failed_entries.append({
                    'Year': year,
                    'Artist': row['Artist'],
                    'Title': row['Title'],
                    'Rank': idx
                })

    # Save results
    billboard_df.to_csv(f"hot100_{year}.csv")
    elapsed = time.time() - start_time
    print(f"✅ Saved {year} data in {elapsed:.1f}s - {len(failed_entries)} missing")

    return billboard_df, failed_entries

async def main(years, concurrency=30):
    """Run the full scraping process with caching and concurrency"""
    cache = LyricsCache()
    all_data = {}
    all_failed_entries = []

    # Reusable HTTP session with connection pooling
    connector = aiohttp.TCPConnector(limit=concurrency)
    async with aiohttp.ClientSession(connector=connector) as session:
        for year in years:
            try:
                df, failed = await process_year(year, cache, session, concurrency)
                if df is not None:
                    all_data[year] = df
                    all_failed_entries.extend(failed)
            except Exception as e:
                print(f"🚨 Critical error processing {year}: {str(e)}")
                logging.exception(e)

    # Final cache save
    cache.save_cache()

    # Save failure log
    if all_failed_entries:
        fail_df = pd.DataFrame(all_failed_entries)
        fail_df.to_csv('missing_lyrics_log.csv', index=False)
        print(f"\nSaved {len(fail_df)} missing entries to log")

    return all_data

# Run on Colab
if __name__ == "__main__":
    # Years to process
    years_to_scrape = list(range(1959, 2025))

    # Run with high concurrency
    hot_100_data = asyncio.run(main(years_to_scrape, concurrency=40))

    # Show summary
    for year, df in hot_100_data.items():
        missing = df[df['Lyrics'] == '']
        print(f"{year}: {len(missing)} missing lyrics")


Processing 1959
Found 100 songs for 1959
🚀 0/100 lyrics from cache


Fetching 1959 lyrics: 100%|██████████| 100/100 [00:11<00:00,  8.58song/s]


✅ Saved 1959 data in 11.9s - 42 missing

Processing 1960
Found 100 songs for 1960
🚀 1/100 lyrics from cache


Fetching 1960 lyrics: 100%|██████████| 99/99 [00:08<00:00, 11.67song/s]


✅ Saved 1960 data in 8.7s - 36 missing

Processing 1961
Found 100 songs for 1961
🚀 0/100 lyrics from cache


...

✅ Saved 2022 data in 6.2s - 5 missing

Processing 2023
Found 100 songs for 2023
🚀 13/100 lyrics from cache


Fetching 2023 lyrics: 100%|██████████| 87/87 [00:06<00:00, 14.23song/s]


✅ Saved 2023 data in 6.4s - 8 missing

Processing 2024
Found 100 songs for 2024
🚀 18/100 lyrics from cache


Fetching 2024 lyrics: 100%|██████████| 82/82 [00:05<00:00, 14.26song/s]


✅ Saved 2024 data in 6.0s - 5 missing

Saved 1088 missing entries to log
1959: 42 missing lyrics
1960: 36 missing lyrics
1961: 29 missing lyrics
1962: 29 missing lyrics
1963: 42 missing lyrics
1964: 40 missing lyrics
1965: 29 missing lyrics
1966: 42 missing lyrics
1967: 33 missing lyrics
1968: 31 missing lyrics
1970: 32 missing lyrics
1971: 42 missing lyrics
1972: 31 missing lyrics
1973: 28 missing lyrics
1974: 22 missing lyrics
1975: 28 missing lyrics
1976: 20 missing lyrics
1977: 22 missing lyrics
1978: 10 missing lyrics
1979: 23 missing lyrics
1980: 22 missing lyrics
1981: 21 missing lyrics
1982: 24 missing lyrics
1983: 10 missing lyrics
1984: 15 missing lyrics
1985: 15 missing lyrics
1986: 9 missing lyrics
1987: 9 missing lyrics
1988: 9 missing lyrics
1989: 12 missing lyrics
1990: 8 missing lyrics
1991: 16 missing lyrics
1992: 9 missing lyrics
1993: 18 missing lyrics
1994: 10 missing lyrics
1995: 17 missing lyrics
1996: 17 missing lyrics
1997: 17 missing lyrics
1998: 14 missing lyr

## 🗜️ Zipping Billboard Hot 100 CSV Files
The function **zip_csv_files()** automates the process of compressing individual Billboard Hot 100 CSV files into a single ZIP archive named hot100_data.zip. It checks for the existence of each yearly CSV file (e.g., hot100_2000.csv) and adds it to the archive if found.

The zipped file is useful for efficiently downloading and transferring the complete dataset between different stages of the project (e.g., from the Data Collection stage to pre-processing and analysis).



In [ ]:

def zip_csv_files(start_year=1959, end_year=2024, output_zip='hot100_data_all_years.zip'):
    with zipfile.ZipFile(output_zip, 'w') as zipf:
        for year in range(start_year, end_year + 1):
            filename = f"hot100_{year}.csv"
            if os.path.exists(filename):
                zipf.write(filename)
    print(f"✅ All CSV files zipped to {output_zip}")

# Create zip file
zip_csv_files()

✅ All CSV files zipped to hot100_data_all_years.zip
